# ETL da camada bronze para camada silver

Este notebook realiza o ETL dos dados da camada bronze para a camada silver. Ou seja: ele abre o dataset e o salva num dataframe, realiza a transformação dos dados e os carrega num arquivo `.csv` e no banco de dados.

### Estratégia de Leitura Otimizada (Chunking)

Para contornar as limitações de memória do ambiente WSL ao ler o arquivo `MICRODADOS_ENEM_2021.csv` (>1GB), adotamos a estratégia de leitura em lotes (*chunks*).
Diferente do padrão do Pandas que tenta carregar todo o arquivo na RAM, este método cria um iterador que processa o arquivo em fragmentos sequenciais, similar ao comportamento "lazy" do Apache Spark.

**Parâmetros Críticos:**
* `chunksize=100000`: Limita o consumo de RAM processando apenas 100 mil linhas por vez.
* `encoding='ISO-8859-1'`: Corrige erros de decodificação de caracteres que contem dentro do arquivo `MICRODADOS_ENEM_2021.csv` que e um "erro" bem comuns em dados  usado no Brasil.
* `sep=';'`: Define o ponto e vírgula como separador correto, evitando que o dataset seja lido em uma única coluna e assim trazendo como esperamos.

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

data_layer_filepath = '../raw/'

chunk_size = 100000

chunks = pd.read_csv(
    data_layer_filepath + 'data_raw/MICRODADOS_ENEM_2021.csv', 
    low_memory=False, 
    encoding='ISO-8859-1', 
    sep=';', 
    chunksize=chunk_size
)

print("Leitura em chunks iniciada...")

contagem_total_linhas = 0

for df_chunk in chunks:
    print(df_chunk.head())
    
    contagem_total_linhas += len(df_chunk)

print(f"Processamento finalizado.")

Leitura em chunks iniciada...
   NU_INSCRICAO  NU_ANO  TP_FAIXA_ETARIA TP_SEXO  TP_ESTADO_CIVIL  \
0  210053865474    2021                5       F                1   
1  210052384164    2021               12       M                1   
2  210052589243    2021               13       F                3   
3  210052128335    2021                3       M                1   
4  210051353021    2021                2       F                1   

   TP_COR_RACA  TP_NACIONALIDADE  TP_ST_CONCLUSAO  TP_ANO_CONCLUIU  TP_ESCOLA  \
0            1                 1                1                3          1   
1            1                 1                1               11          1   
2            1                 1                1               15          1   
3            3                 1                2                0          2   
4            3                 1                2                0          2   

   ...  Q016  Q017  Q018 Q019  Q020 Q021  Q022  Q023  Q024  Q025  
0